In [ ]:
using Gridap, Gridap.Geometry, Gridap.Fields
using Gmsh, GridapGmsh
using GLMakie, DelimitedFiles, Interpolations
using LinearAlgebra, SparseArrays, KrylovKit
using ChainRulesCore, Zygote
using PartitionedArrays
using NLopt
using GridapMakie

import Gridap.CellData: Interpolable
import ChainRulesCore: rrule
import Gmsh: gmsh


main_path = "/Users/jayyao/Documents/Research/RamanInFluid/"
include(main_path*"Materials/Materials.jl")
include(main_path*"Module/Mesh_RecCir.jl")
include(main_path*"Module/Helper.jl")
include(main_path*"Module/GridapFE.jl")
include(main_path*"Module/Control.jl")
include(main_path*"Module/Model.jl")
include(main_path*"Module/Objective.jl")
include(main_path*"Module/Objective_single.jl")

In [ ]:
L = 600
h1 = 600
h2 = 200
rd = 100
rs = 10
rt = 150
dpml = 300

res = 30
l1 = L/res
l2 = l1/10

hrd = (0, h1/2)
meshfile = "geometry.msh"
geo_param = RecCirGeometry(L, h1, h2, rt, rd, rs, dpml, l1, l2)
MeshGenerator(geo_param, meshfile)

In [ ]:
############  Optimization parameters #############
flag_f = true       # Turn on filter
flag_t = true       # Turn on threshold

# Filter and threshold paramters
r = (5, 5)  # Filter radius
β = 32.0                  # β∈[1,∞], threshold sharpness
η = 0.5                   # η∈[0,1], threshold center

α = 0.0 / (2 * 1000.0)    # Equivalent loss α = 1/2Q

# Number of subspace
K = 20

# Amplify g for NLopt
Amp = 1

# Sum over kx
nkx = 30
nparts = nkx / 2

Bp = false          # Matrix B depend on parameters?
pv = 1

# Foundary constraint parameters
c = 625
lw = r[1]
ls = r[1]
ηe = 0.75#fηe(lw / r[1])
ηd = 0.25#fηd(lw / r[1])


control = ControllingParameters(flag_f, flag_t, r, β, η, α, nparts, nkx, K, Amp, Bp, pv, c, ηe, ηd, hrd)

gridap = GridapFE(meshfile, 1, 2, ["DirichletEdges", "DirichletNodes"], ["DesignNodes", "DesignEdges"], ["Target"], ["Source"], flag_f)



In [ ]:

material = "Ag"
n_λ, k_λ = RefractiveIndex(material,main_path,true)
λ1 = 532
λ2 = 582
nm1 = n_λ(λ1) + 1im * k_λ(λ1)
nm2 = n_λ(λ2) + 1im * k_λ(λ2)
nf = 1
μ = 1
R = 1e-10
LHp=(L/2, h1+h2)   # Start of PML for x,y > 0
LHn=(L/2, 0)       # Start of PML for x,y < 0


ω1 = 2 * π / λ1
phys1 = PhysicalParameters(ω1, nf, nm1, nf, μ, R, dpml, LHp, LHn, 0)
ω2 = 2 * π / λ2
phys2 = PhysicalParameters(ω2, nf, nm2, nf, μ, R, dpml, LHp, LHn, 0)

nm1,nm2
# N = num_free_dofs(gridap.FE_U)
# kb = 0. * ω1
# p0 = ones(gridap.np)
# pf_vec = pf_p0(p0; control, gridap)
# pfh = FEFunction(gridap.FE_Pf, pf_vec)
# pth = (pf -> Threshold(pf; control)) ∘ pfh

# A1_mat = MatrixA(pth, kb; phys=phys1, control, gridap)

# binc(v) = ∫(v)gridap.dΓ_s
# b_inc = assemble_vector(binc, gridap.FE_V)

# u1_vec = A1_mat\b_inc
# u1h = FEFunction(gridap.FE_U, u1_vec)

# A2_mat = MatrixA(pth, kb; phys=phys2, control, gridap)
# O_mat = MatrixOc(ω2, nf^2; gridap)

# braman(v) = ∫((x->GaussianD(x, hrd, [2,2]))*(∇(v) ⋅ ∇(u1h)))gridap.dΩ
# b_raman = assemble_vector(braman, gridap.FE_V)
# v2_vec = A2_mat' \ b_raman
# v2h = FEFunction(gridap.FE_U, v2_vec)

# @show gvalue = v2_vec'*O_mat*v2_vec

# #fig, ax, plt = plot(gridap.Ω, pth, colormap = :binary)
# maxv = maximum(abs.(u1_vec))/1
# fig, ax, plt = plot(gridap.Ω, real(u1h), colormap = :bwr, colorrange=(-maxv, maxv))
# Colorbar(fig[1,2], plt)
# ax.title = "Design Shape"
# ax.aspect = AxisAspect((L+2*dpml)/(h1+h2+2*dpml))
# #ax.aspect = AxisAspect(L/(h1))
# #limits!(ax, -L/2, L/2, 0, h1)
# fig
#0.12931555555555557 + 3.1932im

In [ ]:
using FileIO
import Images: Gray
# specify the path to your local image file
img_path = "Initial/Rasmus_Focus.png"
img = load(img_path)
data = 1.0 .-Float64.(Gray.(img))
function image_to_function(x, data, Lx, Ly)
    Nx, Ny = size(data)
    #xi = Int(round(((x[1]+3)/Lx + 0.5) * Nx))
    #yi = Ny-Int(round(((x[2]-294)/Ly + 0.5) * Ny))
    xi = Int(round(((x[1])/Lx + 0.5) * Nx))
    yi = Ny-Int(round(((x[2]-300)/Ly + 0.5) * Ny))
    if xi > 0 && xi <= Nx && yi > 0 && yi <= Ny
        return data[xi,yi]
    else
        return 0.0
    end
end

image_size = 281
binit(v) = ∫(v * x->image_to_function(x, data, image_size, image_size))gridap.dΩ
pc_vec = assemble_vector(binit, gridap.FE_P)
p_init = p_extract(pc_vec; gridap)
p_init[p_init .< 0.1] .= 0
p_init[p_init .> 0.1] .= 1

# p_init = ones(gridap.np)
pf_vec = pf_p0(p_init; control, gridap)
pfh = FEFunction(gridap.FE_Pf, pf_vec)
pth = (pf -> Threshold(pf; control)) ∘ pfh

@show gf_p(p_init, []; kb=0, phys1, control, gridap)
fig, ax, plt = plot(gridap.Ω, pth, colormap = :binary)
Colorbar(fig[1,2], plt)
ax.aspect = AxisAspect(L/(h1+h2))
ax.title = "Design Shape"
limits!(ax, -L/2, L/2, 0, h1+h2)
# save("Results/shape.png", fig)
fig

In [ ]:
# p_init = ones(gridap.np) * 0.5 + rand(gridap.np) * 1e-2
# β_list = [5.0, 10.0, 20.0, 30.0, 40.0, 60.0, 80.0]
β_list = [8.0, 8.0, 16.0, 16.0, 32.0, 32.0, 32.0]

g_opt = 0
for bi = 1 : 7
    β = β_list[bi]
    control = ControllingParameters(flag_f, flag_t, r, β, η, α, nparts, nkx, K, Amp, Bp, pv, c, ηe, ηd, hrd)

#     if bi == 1
#         g_opt, p_opt = gs_p_optimize(p_init, 1e-12, 100; phys1, phys2, control, gridap)
    
#     else
#         g_opt, p_opt = gs_p_optimize([], 1e-12, 100; phys1, phys2, control, gridap)
#     end
    
    if bi == 1
        g_opt, p_opt = gf_p_optimize(p_init, 1e-12, 100; phys1, control, gridap)
    
    else
        g_opt, p_opt = gf_p_optimize([], 1e-12, 100; phys1, control, gridap)
    end
    if isfile("p_opt.value.txt")
        run(`rm p_opt_value.txt`)
    end
    open("p_opt_value.txt", "w") do iop
        for i = 1 : length(p_opt)
            p_temp = p_opt[i]
            write(iop, "$p_temp \n")
        end
    end
    open("g_opt_value.txt", "a") do io
        write(io, "$g_opt \n")
    end
end
@show g_opt

In [ ]:
# Output and view
ImportFromFile = true
N = num_free_dofs(gridap.FE_U)
if ImportFromFile
    #p_max = readdlm("cloud/task5/p_opt_value.txt", Float64)
    p_max = readdlm("p_opt_value.txt", Float64)
    p0 = p_max[:]
else
    p0 = zeros(gridap.np)
end

kb = 0
pf_vec = pf_p0(p0; control, gridap)
pfh = FEFunction(gridap.FE_Pf, pf_vec)
pth = (pf -> Threshold(pf; control)) ∘ pfh
A1_mat = MatrixA(pth, kb; phys=phys1, control, gridap)
b1_vec = assemble_vector(v->(∫(v)gridap.dΓ_s), gridap.FE_V)
u1_vec = A1_mat \ b1_vec
u1h = FEFunction(gridap.FE_U, u1_vec)

# A2_mat = MatrixA(pth, kb; phys=phys2, control, gridap)
# o_vec = VectorO(1, 1; gridap)
# v2_vec = A2_mat' \ o_vec
# v2h = FEFunction(gridap.FE_U, v2_vec)

fig, ax, plt = plot(gridap.Ω, pth, colormap = :binary)
Colorbar(fig[1,2], plt)
ax.aspect = AxisAspect(L/(h1+h2))
ax.title = "Design Shape"
limits!(ax, -L/2, L/2, 0, h1+h2)
save("Results/shape.png", fig)
fig

In [ ]:
maxv = maximum(abs.(u1_vec))/1
#fig, ax, plt = plot(gridap.Ω, real(u1h), colormap = :bwr, colorrange=(-maxv, maxv))
fig, ax, plt = plot(gridap.Ω, abs2(u1h), colormap = :hot, colorrange=(0, maxv^2/1))
Colorbar(fig[1,2], plt)
ax.title = "H field"
ax.aspect = AxisAspect((L)/(h1+h2))
#ax.aspect = AxisAspect(L/(h1))
limits!(ax, -L/2, L/2, 0, h1+h2)
fig

In [ ]:
p0 = zeros(gridap.np)
δp = rand(gridap.np)*1e-8
grad = zeros(gridap.np)

kb = 0.0 * ω1
g0 = gs_p(p0, grad; kb, phys1, phys2, control, gridap)
g1 = gs_p(p0+δp, []; kb, phys1, phys2, control, gridap)
# g0 = gf_p(p0, grad; kb, phys1, control, gridap)
# g1 = gf_p(p0+δp, []; kb, phys1, control, gridap)
# g0 = LSConstraint(p0, grad; control, gridap)
# g1 = LSConstraint(p0+δp, []; control, gridap)
g0,g1-g0, grad'*δp

In [ ]:
4.8/2.5480895094821322e-5

In [ ]:
gf_p(zeros(gridap.np), []; kb=0, phys1, control, gridap)

In [ ]:
4734/1796